In [47]:
import numpy as np

Neural networks with multiple layers are just nested functions! i.e. each layer has a function that maps the input from one domain to the other.

$\Large y = h(g(f(x)))$

$\Large a = f(x)$

$\Large b = g(a) $

$\Large c = h(b) $

We can compute the final derivative of $y$ w.r.t $x$ with the Jacobian:
$\Large\frac{\partial y}{\partial x} = \frac{\partial h(b)}{\partial b} \frac{\partial g(a)}{\partial a} \frac{\partial f(x)}{\partial x}$

To simplify:

$\Large C \,=\, \frac{\partial h(b)}{\partial b}$

$\Large B \,=\, \frac{\partial g(a)}{\partial a}$

$\Large A \,=\, \frac{\partial f(x)}{\partial x}$



So we just need to multiply these 3 matrices. But in which order?

### Associative property makes them equivalent, but numerically, the costs are different!

In [48]:
ysize = 1; # Set as 1 for ysize > xsize, or 100
m = 150;
n = 150;
xsize = 100; # Set as 1 for xsize > ysize or 100

A = np.random.randn(n,xsize);
B = np.random.randn(m,n);
C = np.random.randn(ysize,m);

In [49]:
print('A shape:', A.shape)
print('B shape:', B.shape)
print('C shape:', C.shape)

A shape: (150, 1)
B shape: (150, 150)
C shape: (100, 150)


## Why do the matrix sizes matter? 
The order of matrix computations in $C, B,  A$ determine computational cost.

As an experiment, Try two conditions a) ysize > xsize, b) ysize < xsize. 

We will see that depending on these conditions, either forward or reverse mode can result in fewer computations = Good for computer memory, efficiency and time! This operation would be performed typically several hundreds of thousands of times in a deep neural network training (if not millions to billions), so we need the optimal method to calculate.

## Forward Mode

i.e. $\frac{\partial y}{\partial x} = C \cdot (B \cdot A)$

In [50]:
intermediate_FWD = np.dot(B,A) # B . A
print('Intermediate Derivative shape:', intermediate_FWD.shape)

Intermediate Derivative shape: (150, 1)


In [51]:
dydx_FWD = np.dot(C,intermediate_FWD)
print('Final derivative shape:', dydx_FWD.shape)

Final derivative shape: (100, 1)


In [52]:
num_computations_FWD = (xsize * n * m) + (ysize * m * xsize)
print('Number of scalar multiplications in FORWARD mode:', num_computations_FWD)

Number of scalar multiplications in FORWARD mode: 37500


## Reverse Mode

i.e. $\frac{\partial y}{\partial x} = (C \cdot B) \cdot A$

In [53]:
intermediate_REV = np.dot(C,B) # C . B
print('Intermediate Derivative shape:', intermediate_REV.shape)

Intermediate Derivative shape: (100, 150)


In [54]:
dydx_REV = np.dot(intermediate_REV,A)
print('Final derivative shape:', dydx_REV.shape)

Final derivative shape: (100, 1)


In [55]:
num_computations_REV = (ysize * m * n) + (ysize * n * xsize)
print('Number of scalar multiplications in REVERSE mode:', num_computations_REV)

Number of scalar multiplications in REVERSE mode: 2265000


In [56]:
if num_computations_FWD < num_computations_REV:
    print('Forward Mode is cheaper!')
elif num_computations_REV < num_computations_FWD:
    print('Reverse Mode is cheaper!')

Forward Mode is cheaper!


## In training deep neural networks, usually y < x i.e. Reverse mode is almost always preferred

Reverse mode is suitable when the function has MANY inputs, but FEWER outputs. The output when training a neural net is a **scalar valued** loss, while the input is a matrix or a tensor (images, volumetric data etc.)

Deep learning libraries perform backpropagation in reverse mode, though there are situations in scientific computing where you would prefer forward mode - you can do the simple exercise above to know what is optimal for your problem.